In [28]:
# Diamond Company Analytics Challenge — Polars Solution
# ----------------------------------------------------
import polars as pl

print("Loading diamonds dataset...")
df = pl.read_csv(
    "https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/diamond.csv",
    infer_schema_length=2000
)
print("Dataset loaded successfully!")
print("-" * 72)

# (Helper) Safe column access with friendly error if schema differs
def require_cols(cols):
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise ValueError(
            f"Expected columns missing from dataset: {missing}\n"
            f"Available columns: {df.columns}"
        )

# Try to align to the classic diamonds schema if possible
# Expected core columns for this exercise:
core_cols = ["Carat Weight","Cut","Color","Clarity","Price"]
require_cols(core_cols)

# -----------------------------
# Section 1: Initial Assessment
# -----------------------------
print("Section 1: Initial Inventory Assessment")
print()

# Q1: Scale of the database
n_rows, n_cols = df.height, df.width
print(f"Q1 — Inventory scale: {n_rows:,} rows (diamonds), {n_cols} columns (attributes).")

Loading diamonds dataset...
Dataset loaded successfully!
------------------------------------------------------------------------
Section 1: Initial Inventory Assessment

Q1 — Inventory scale: 6,000 rows (diamonds), 8 columns (attributes).


In [29]:
# Q2 Business term → Code: "What columns" → df.columns
print("Q2:", df.columns)


Q2: ['Carat Weight', 'Cut', 'Color', 'Clarity', 'Polish', 'Symmetry', 'Report', 'Price']


#Q3
Each record describes an individual diamond, including physical characteristics (carat, cut, color, clarity, dimensions) and market value.

In [30]:
#Q4
total_carats = df.select(pl.sum("Carat Weight")).item()

print("=== Total Carats in Inventory ===")
print(f"Total carats: {total_carats:,.2f}")

=== Total Carats in Inventory ===
Total carats: 8,007.12


In [31]:
# Q5: Variety of cuts (distinct styles, not quantities)
print("\nQ5 — Variety of cuts (distinct styles we carry):")
n_cuts = df.select(pl.col("Cut").n_unique()).item()
cuts_df = df.select(pl.col("Cut").unique().sort().alias("cuts"))
print(f"Number of distinct cuts: {n_cuts}")
print("Cuts:")
print(cuts_df)


Q5 — Variety of cuts (distinct styles we carry):
Number of distinct cuts: 5
Cuts:
shape: (5, 1)
┌─────────────────┐
│ cuts            │
│ ---             │
│ str             │
╞═════════════════╡
│ Fair            │
│ Good            │
│ Ideal           │
│ Signature-Ideal │
│ Very Good       │
└─────────────────┘


In [32]:
# Q6: Most and least valuable diamond (by listed Price)

print("\nQ6 — Price exposure (most & least valuable diamonds):")

most_val = (
    df.select(core_cols)
      .sort("Price", descending=True)  # <- cross-version safe
      .head(1)
)
least_val = (
    df.select(core_cols)
      .sort("Price")                   # ascending (least valuable)
      .head(1)
)

print("Most valuable diamond:")
print(most_val)
print("Least valuable diamond:")
print(least_val)


Q6 — Price exposure (most & least valuable diamonds):
Most valuable diamond:
shape: (1, 5)
┌──────────────┬───────┬───────┬─────────┬────────┐
│ Carat Weight ┆ Cut   ┆ Color ┆ Clarity ┆ Price  │
│ ---          ┆ ---   ┆ ---   ┆ ---     ┆ ---    │
│ f64          ┆ str   ┆ str   ┆ str     ┆ i64    │
╞══════════════╪═══════╪═══════╪═════════╪════════╡
│ 2.79         ┆ Ideal ┆ D     ┆ IF      ┆ 101561 │
└──────────────┴───────┴───────┴─────────┴────────┘
Least valuable diamond:
shape: (1, 5)
┌──────────────┬──────┬───────┬─────────┬───────┐
│ Carat Weight ┆ Cut  ┆ Color ┆ Clarity ┆ Price │
│ ---          ┆ ---  ┆ ---   ┆ ---     ┆ ---   │
│ f64          ┆ str  ┆ str   ┆ str     ┆ i64   │
╞══════════════╪══════╪═══════╪═════════╪═══════╡
│ 0.77         ┆ Good ┆ I     ┆ VS1     ┆ 2184  │
└──────────────┴──────┴───────┴─────────┴───────┘


In [33]:
# Q7: Typical prices across color grades (use median as 'typical')
print("\nQ7 — Typical (median) price by color grade:")
q7 = (
    df.group_by("Color")
      .agg([
          pl.len().alias("n_diamonds"),
          pl.col("Price").median().alias("median_price"),
          pl.col("Price").mean().alias("avg_price"),
      ])
      .sort("Color")
)
print(q7)

print("\nReflection after Q7:")
print("  New grain = one row per Color grade (an aggregated color segment of inventory).")


Q7 — Typical (median) price by color grade:
shape: (6, 4)
┌───────┬────────────┬──────────────┬──────────────┐
│ Color ┆ n_diamonds ┆ median_price ┆ avg_price    │
│ ---   ┆ ---        ┆ ---          ┆ ---          │
│ str   ┆ u32        ┆ f64          ┆ f64          │
╞═══════╪════════════╪══════════════╪══════════════╡
│ D     ┆ 661        ┆ 7567.0       ┆ 15255.783661 │
│ E     ┆ 778        ┆ 7059.5       ┆ 11539.190231 │
│ F     ┆ 1013       ┆ 8860.0       ┆ 12712.241856 │
│ G     ┆ 1501       ┆ 8800.0       ┆ 12520.050633 │
│ H     ┆ 1079       ┆ 7169.0       ┆ 10487.347544 │
│ I     ┆ 968        ┆ 6655.0       ┆ 8989.636364  │
└───────┴────────────┴──────────────┴──────────────┘

Reflection after Q7:
  New grain = one row per Color grade (an aggregated color segment of inventory).


In [34]:
# Q8: Clarity breakdown — count, typical (median) price, total carats heldprint("\nQ8 — Inventory by clarity grade (count, typical price, total carats):")
q8 = (
    df.group_by("Clarity")
      .agg([
          pl.len().alias("n_diamonds"),
          pl.col("Price").median().alias("median_price"),
          pl.col("Carat Weight").sum().alias("total_carats"),
      ])
      .sort("Clarity")
)
print(q8)

shape: (7, 4)
┌─────────┬────────────┬──────────────┬──────────────┐
│ Clarity ┆ n_diamonds ┆ median_price ┆ total_carats │
│ ---     ┆ ---        ┆ ---          ┆ ---          │
│ str     ┆ u32        ┆ f64          ┆ f64          │
╞═════════╪════════════╪══════════════╪══════════════╡
│ FL      ┆ 4          ┆ 62371.5      ┆ 7.87         │
│ IF      ┆ 219        ┆ 12647.0      ┆ 316.0        │
│ SI1     ┆ 2059       ┆ 5417.0       ┆ 2563.69      │
│ VS1     ┆ 1192       ┆ 9245.0       ┆ 1682.74      │
│ VS2     ┆ 1575       ┆ 7568.0       ┆ 2170.81      │
│ VVS1    ┆ 285        ┆ 11142.0      ┆ 389.43       │
│ VVS2    ┆ 666        ┆ 10266.0      ┆ 876.58       │
└─────────┴────────────┴──────────────┴──────────────┘


In [35]:
# Q9 — Pricing structure across Cut × Color
print("\nQ9 — Pricing structure across Cut × Color:")

q9 = (
    df.group_by(["Cut", "Color"])
      .agg([pl.len().alias("n"), pl.col("Price").median().alias("med"), pl.col("Price").mean().alias("avg")])
      .sort(["Cut", "Color"])
)
print(q9)

print("\nQ9 (matrix) — Median price by Cut × Color:")
q9_pivot = df.pivot(values="Price", index="Cut", on="Color", aggregate_function="median").sort("Cut")
print(q9_pivot)

print("\nReflection after Q9:")
print("  New grain = one row = a single Cut + Color pair.")


Q9 — Pricing structure across Cut × Color:
shape: (30, 5)
┌───────────┬───────┬─────┬────────┬──────────────┐
│ Cut       ┆ Color ┆ n   ┆ med    ┆ avg          │
│ ---       ┆ ---   ┆ --- ┆ ---    ┆ ---          │
│ str       ┆ str   ┆ u32 ┆ f64    ┆ f64          │
╞═══════════╪═══════╪═════╪════════╪══════════════╡
│ Fair      ┆ D     ┆ 12  ┆ 5040.0 ┆ 6058.25      │
│ Fair      ┆ E     ┆ 32  ┆ 4908.5 ┆ 5370.625     │
│ Fair      ┆ F     ┆ 24  ┆ 4628.0 ┆ 6063.625     │
│ Fair      ┆ G     ┆ 21  ┆ 4646.0 ┆ 7345.52381   │
│ Fair      ┆ H     ┆ 24  ┆ 4358.0 ┆ 5908.5       │
│ …         ┆ …     ┆ …   ┆ …      ┆ …            │
│ Very Good ┆ E     ┆ 323 ┆ 6703.0 ┆ 12101.910217 │
│ Very Good ┆ F     ┆ 455 ┆ 8330.0 ┆ 12413.905495 │
│ Very Good ┆ G     ┆ 578 ┆ 8249.0 ┆ 12354.013841 │
│ Very Good ┆ H     ┆ 424 ┆ 6564.0 ┆ 10056.106132 │
│ Very Good ┆ I     ┆ 383 ┆ 6684.0 ┆ 8930.031332  │
└───────────┴───────┴─────┴────────┴──────────────┘

Q9 (matrix) — Median price by Cut × Color:
shape: (5, 7)

In [36]:
#Q10
print("\nQ10 — Top 5 most valuable Color–Cut segments by total inventory value:")
q10 = (
    df.group_by(["Color","Cut"])
      .agg([
          pl.len().alias("n_diamonds"),
          pl.col("Price").sum().alias("total_value"),
          pl.col("Carat Weight").sum().alias("total_carats"),
          pl.col("Price").median().alias("median_price"),
      ])
      .sort("total_value", descending=True)
      .head(5)
)
print(q10)


Q10 — Top 5 most valuable Color–Cut segments by total inventory value:
shape: (5, 6)
┌───────┬───────────┬────────────┬─────────────┬──────────────┬──────────────┐
│ Color ┆ Cut       ┆ n_diamonds ┆ total_value ┆ total_carats ┆ median_price │
│ ---   ┆ ---       ┆ ---        ┆ ---         ┆ ---          ┆ ---          │
│ str   ┆ str       ┆ u32        ┆ i64         ┆ f64          ┆ f64          │
╞═══════╪═══════════╪════════════╪═════════════╪══════════════╪══════════════╡
│ G     ┆ Ideal     ┆ 690        ┆ 9363514     ┆ 966.9        ┆ 9510.5       │
│ G     ┆ Very Good ┆ 578        ┆ 7140620     ┆ 795.4        ┆ 8249.0       │
│ F     ┆ Very Good ┆ 455        ┆ 5648327     ┆ 592.77       ┆ 8330.0       │
│ F     ┆ Ideal     ┆ 363        ┆ 5346782     ┆ 502.01       ┆ 10360.0      │
│ H     ┆ Ideal     ┆ 458        ┆ 5279687     ┆ 653.15       ┆ 8014.0       │
└───────┴───────────┴────────────┴─────────────┴──────────────┴──────────────┘
